In [1]:
import pandas as pd
import sys
import jieba

In [2]:
sys.path.append('../utils/')
sys.path.append('../feature/')
import os 
import pandas as pd
import numpy as np
from collections import defaultdict


import config
from w2v import load_my_train_w2v,load_pre_train_w2v
from CutWord import cut_word,more
from feat_gen import *

if config.use_pre_train:
    vocab, embed_weights = load_pre_train_w2v(config.origin_csv)
else:
    vocab, embed_weights = load_my_train_w2v(config.origin_csv)


from help import score

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.405 seconds.
Prefix dict has been built succesfully.


IOError: [Errno 2] No such file or directory: 'data/share/jieba/jieba_dict.txt'

In [12]:
data = pd.read_hdf('../data/cache/train_cut_char.hdf')
# data['q1_cut'] = data['q1'].map(lambda x: jieba.cut(x.strip()))
# data['q2_cut'] = data['q2'].map(lambda x:jieba.cut(x.strip()))

In [13]:
data.head()

,id,q1,q2,label,q1_cut,q2_cut
0,1,﻿怎么更改花呗手机号码,我的花呗是以前的手机号码，怎么更改成现在的支付宝的号码手机号,1,"[﻿, 怎, 么, 更, 改, 花, 呗, 手, 机, 号, 码]","[我, 花, 呗, 是, 以, 前, 手, 机, 号, 码, 怎, 么, 更, 改, 成, ..."
1,2,也开不了花呗，就这样了？完事了,真的嘛？就是花呗付款,0,"[也, 开, 不, 花, 呗, 就, 这, 样, 完, 事]","[真, 嘛, 就, 是, 花, 呗, 付, 款]"
2,3,花呗冻结以后还能开通吗,我的条件可以开通花呗借款吗,0,"[花, 呗, 冻, 结, 以, 后, 还, 能, 开, 通]","[我, 条, 件, 可, 以, 开, 通, 花, 呗, 借, 款]"
3,4,如何得知关闭借呗,想永久关闭借呗,0,"[如, 何, 得, 知, 关, 闭, 借, 呗]","[想, 永, 久, 关, 闭, 借, 呗]"
4,5,花呗扫码付钱,二维码扫描可以用花呗吗,0,"[花, 呗, 扫, 码, 付, 钱]","[二, 维, 码, 扫, 描, 可, 以, 用, 花, 呗]"


In [17]:
import numpy as np
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis

In [15]:
pretrained_weights = np.load('../data/share/my_w2v/word_embed_weight_char.npy')

In [9]:
pretrained_weights

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.41712812, -0.25747097,  1.34245288, ...,  0.14645386,
        -0.41550475, -0.13932288],
       [-0.16524845,  0.15986773, -0.75437713, ..., -0.71025795,
         0.51971549, -0.58849168],
       ..., 
       [ 0.03536939,  0.00969836, -0.02836092, ...,  0.02810706,
         0.0015696 ,  0.00739182],
       [ 0.01173481,  0.02284247, -0.02278483, ..., -0.01134644,
        -0.00295069,  0.03615419],
       [ 0.18389269, -0.2051181 ,  0.1962227 , ..., -0.01302521,
         0.09344642, -0.09769231]])

In [219]:
data['q1_len'] = data.q1.map(lambda x:len(x))
data['q2_len'] = data.q2.map(lambda x:len(x))

In [222]:
data[['q1_len','q2_len']].describe()

,q1_len,q2_len
count,102477.000000,102477.000000
mean,13.390956,13.404520
std,6.111260,6.134927
min,5.000000,3.000000
25%,10.000000,10.000000
50%,12.000000,12.000000
75%,15.000000,15.000000
max,97.000000,112.000000


In [143]:

def dif(row):
    q1 = set(row[u'q1_cut'])
    q2 = set(row[u'q2_cut'])
    return q1.difference(q2)
def dif2(row):
    q1 = set(row[u'q1_cut'])
    q2 = set(row[u'q2_cut'])
    return q2.difference(q1)


data['char_df1']=data.apply(lambda x:dif(x),axis=1)
data['char_df2']=data.apply(lambda x:dif2(x),axis=1)
# def zero12(row):
#     return len(row['char_df1'])==0 or len(row['char_df2'])==0)
# def zero21(row):
#     return int(len(row['char_df1'])==0 or len(row['char_df2'])==0)
data['zero1'] = data['char_df1'].map(lambda x:len(x))
data['zero2'] = data['char_df2'].map(lambda x:len(x))

In [146]:
data['len_x1'] = data['q1_cut'].map(lambda x:len(list(x)))
data['len_x2'] = data['q2_cut'].map(lambda x:len(list(x)))

In [166]:
import re

In [209]:
def clean_str(x):
    #x = re.sub('***',u'X',x)

    punc = "蚂蚁 为何 为什么 怎么 如何 了 吗  的 ！？。 ＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏."
    punc = punc.decode("utf-8")
    return re.sub(ur"[%s]+" %punc, "", x)

In [210]:
data.head(100).q1.map(lambda x: ' '.join(jieba.cut(x.strip())))

0                                   ﻿ 怎么 更改 花 呗 手机号码
1                         也 开 不了 花 呗 ， 就 这样 了 ？ 完事 了
2                                 花 呗 冻结 以后 还 能 开通 吗
3                                       如何 得知 关闭 借 呗
4                                          花 呗 扫码 付钱
5                                   花 呗 逾期 后 不能 分期 吗
6                                          花 呗 分期 清空
7                                       借 呗 逾期 短信 通知
8                        借 呗 即将 到期 要 还 的 账单 还 能 分期 吗
9                                花 呗 为什么 不能 支付 手机 交易
10                       在 吗 ， 双 * * * 有 临时 花 呗 额度 吗
11                                  我 什么 时候 开通 了 花 呗
12             花 呗 每月 还 了 最低 还款 后 还 会 有 万分之 五 的 利息 吗
13                              我 想 用 蚂蚁 借 呗 怎么 用 不了
14                                      花 呗 也 不能 用 了
15                                 花 呗 的 安全 没有 验证 成功
16                                       我 开通 不了 借 呗
17                                借 呗 还款 了 ， 额度 未 恢复
18                     就是 我花 呗 忘记 还款 了 。 逾期 一天

In [211]:
data.head(100).q1.apply(lambda x:clean_str(x)).map(lambda x: ' '.join(jieba.cut(x.strip())))

0                                ﻿ 更改 花 呗 手机号码
1                              也 开不花 呗 就 这样 完事
2                             花 呗 冻结 以后 还 能 开通
3                                    得知 关闭 借 呗
4                                    花 呗 扫码 付钱
5                               花 呗 逾期 后 不能 分期
6                                    花 呗 分期 清空
7                                 借 呗 逾期 短信 通知
8                      借 呗 即将 到期 要 还 账单 还 能 分期
9                              花 呗 不能 支付 手机 交易
10                       在 双 * * * 有 临时 花 呗 额度
11                                 我 时候 开通 花 呗
12             花 呗 每月 还 最低 还款 后 还 会 有 万分之 五 利息
13                               我 想 用 借 呗 用 不
14                                  花 呗 也 不能 用
15                             花 呗 安全 没有 验证 成功
16                                   我 开通 不借 呗
17                              借 呗 还款额 度 未 恢复
18                      就是 我花 呗 忘记 还款 逾期 一天 有事
19                           我 问 你 我 借 呗 要 还 钱
20                          是否 有花 呗 就 不 可以 借 呗
21           

In [173]:
data.head(100).q1_cut.map(lambda x:' '.join(list(x)))

0      
1      
2      
3      
4      
5      
6      
7      
8      
9      
10     
11     
12     
13     
14     
15     
16     
17     
18     
19     
20     
21     
22     
23     
24     
25     
26     
27     
28     
29     
     ..
70     
71     
72     
73     
74     
75     
76     
77     
78     
79     
80     
81     
82     
83     
84     
85     
86     
87     
88     
89     
90     
91     
92     
93     
94     
95     
96     
97     
98     
99     
Name: q1_cut, Length: 100, dtype: object

In [147]:
data.label.value_counts()

0    83792
1    18685
Name: label, dtype: int64

In [148]:
import pandas as pd 
import gc
import xgboost as xgb
from math import floor
from time import time


feats = ['zero1','zero2','len_x1','len_x2']


# Train model
X_train = data[:80000][feats]
print(X_train.shape)
Y_train = data[:80000]['label']

X_valid = data[80000:][feats]
Y_valid =data[80000:]['label']

params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'
params["max_depth"] = 8
params["silent"] = 1
params["seed"] = 1632

d_train = xgb.DMatrix(X_train, label=Y_train)
d_valid = xgb.DMatrix(X_valid, label=Y_valid)
watchlist = [(d_train, 'train'), (d_valid, 'valid')]

print('train first model....')
bst = xgb.train(params, d_train, 50000, watchlist,early_stopping_rounds=150, verbose_eval=10)


# This is used to determine the best round to stop on. This is not the ideal traning method,
# but it does allow training the model on the entire data set, which gave some gains over
# using a only a train/test method + early stopping.
n_iter = bst.best_iteration

pred = bst.predict(d_valid)
pred = [int(x>0.5) for x in pred]
print(sum(pred))
print(sum(Y_valid))
pre, rec, f1 = score(Y_valid, pred)
print('p r f1 ', pre, rec, f1)


(80000, 4)
train first model....
[0]	train-logloss:0.59163	valid-logloss:0.582237
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 150 rounds.
[10]	train-logloss:0.476803	valid-logloss:0.441611
[20]	train-logloss:0.476621	valid-logloss:0.440405
[30]	train-logloss:0.476619	valid-logloss:0.440381
[40]	train-logloss:0.476618	valid-logloss:0.44038
[50]	train-logloss:0.476618	valid-logloss:0.440381
[60]	train-logloss:0.476617	valid-logloss:0.440381
[70]	train-logloss:0.476617	valid-logloss:0.440381
[80]	train-logloss:0.476617	valid-logloss:0.440381
[90]	train-logloss:0.476617	valid-logloss:0.440381
[100]	train-logloss:0.476616	valid-logloss:0.440381
[110]	train-logloss:0.476616	valid-logloss:0.440381
[120]	train-logloss:0.476616	valid-logloss:0.440381
[130]	train-logloss:0.476616	valid-logloss:0.440381
[140]	train-logloss:0.476616	valid-logloss:0.44038
[150]	train-logloss:0.476616	valid-logloss:0.4403

/home/bigdata/anaconda3/envs/py2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/bigdata/anaconda3/envs/py2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
